We start by adding the necessary folders to the current working path.

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add TRA folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/TRA3D")

# add ActiveBarriers folder to current working path
sys.path.append(parent_directory+"/demos/ActiveBarriers")

# remove warnings from shapely due to deprecation
import warnings
warnings.filterwarnings('ignore')

# Overview

We illustrate the numerical implementation and the use of the active Trajectory Rotation Average ($ \mathrm{a\overline{TRA}} $) on a direct numerical simulation of a turbulent channel flow from the publicly available [Johns Hopkins University Turbulence
Database (JHTDB)](http://turbulence.pha.jhu.edu/). The active LCS diagnostic is applied to the three-dimensional and steady instantaneous linear momentum barrier equation. The notebook is structured as follows:

1. Import Laplacian of velocity-field data stored in the folder 'data/Turbulence'. The Laplacian of the velocity field from the publicly available John Hopkins University Turbulence Database has been [precomputed](../../../../data/Turbulence/Channel.ipynb).
<br />
2. Define computational parameters (such as the number of cores) and data.
 <br />
3. Define spatio-temporal domain.
 <br />
4. Interpolate Laplacian of velocity from (discrete) gridded data.
 <br />
5. Instantaneous linear momentum barriers from active Trajectory Rotation Average ($ \mathrm{a\overline{TRA}} $):

   <br />
   * For 3D incompressible, uniform-density Navier–Stokes flows, the Eulerian barrier equation for linear momentum is:
   <br />
   \begin{equation}
   \mathbf{x'}(s) = \nu \rho \Delta \mathbf{v}(\mathbf{x}, t)
   \label{eq: linearmoment1}
   \end{equation}
   
   <br />
   
   By appropriate rescaling with respect to the dummy time, equation \ref{eq: linearmoment1} simplifies to: 
   <br />
   \begin{equation}
   \mathbf{x'}(s) = \Delta \mathbf{v}(\mathbf{x}, t)
   \label{eq: linearmoment2}
   \end{equation}
   
    Time 't' is just a fixed parameter and hence the system is autonomous and the integration is done with respect to a dummy time variable 's'. 's' can be arbitrarily large and the $ \mathrm{a\overline{TRA}} $ is only limited by the spatial resolution and domain. It can thus potentially reveal finer and more resolved structures compared to the classic version of the $ \mathrm{\overline{TRA}} $, which is fundamentally limited by the temporal domain of the available data.
    
    <br />
    
   * We start by evaluating the right-hand-side of equation \ref{eq: linearmoment2} by first interpolating $ \Delta \mathbf{v}(\mathbf{x}, t) $ over the spatial meshgrid.
   
   <br />
   
   * We then apply the $ \mathrm{a\overline{TRA}} $ to the autonomous ODE from equation \ref{eq: linearmoment2} over different dummy time-intervals.
<br />
7. References

# Import data

In [2]:
# import scipy
import scipy.io as sio

# import numpy
import numpy as np

f = sio.loadmat('../../../../data/Turbulence/Lap_velocity_TurbulentChannelFlow.mat')

lap_U = f['Lap_u']
lap_V = f['Lap_v']
lap_W = f['Lap_w']

x = f['xspan']
y = f['yspan']
z = f['zspan']

FileNotFoundError: [Errno 2] No such file or directory: '../../../../data/Turbulence/Lap_velocity_TurbulentChannelFlow.mat'

# Computational parameters and data

Here we define the computational parameters and the data.

In [ ]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 7 # int

# Periodic boundary conditions
periodic_x = False # bool
periodic_y = False # bool
periodic_z = False # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = False # bool

# Defined domain
defined_domain = np.isfinite(lap_U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
# boundaries
xmin = 3.6 # float
xmax = 4 # float
ymin = 0.2 # float
ymax = 0.6 # float
z_section = 2.55 # float

assert np.min(X) < xmin < np.max(X), f'xmin must be between {np.min(X)} and {np.max(X)}'
assert np.min(X) < xmax < np.max(X), f'xmax must be between {np.min(X)} and {np.max(X)}'
assert np.min(Y) < ymin < np.max(Y), f'ymin must be between {np.min(Y)} and {np.max(Y)}'
assert np.min(Y) < ymax < np.max(Y), f'ymax must be between {np.min(Y)} and {np.max(Y)}'
assert np.min(Z) < z_section < np.max(Z), f'z_section must be between {np.min(Z)} and {np.max(Z)}'

# Resolution of meshgrid
Ny = 500 # int
Nx = 500 # int

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True) # array (Nx, )
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True) # array (Ny, )

dx = x_domain[1]-x_domain[0] # float
dy = y_domain[1]-y_domain[0] # float

X_domain, Y_domain = np.meshgrid(x_domain, y_domain) # array (Ny, Nx)

# Define dummy time-intervals for computation of aFTLE in ascending order
s1 = 10**(-4) # float s1 > 0
s2 = 0.5*10**(-3) # float s2 > s1
s3 = 10**(-3) # float s3 > s2

s = [s1, s2, s3] # list (3,)

# define dummy time interval
ds = .5*10**(-5)
sN = s[-1] # Feel free to change this parameter as needed
s_dummy = np.arange(0, sN+ds, ds)

# Laplacian of velocity Interpolation

In order to evaluate the laplacian of the velocity field at arbitrary locations, we interpolate the gridded laplacian of velocity data. The interpolation with respect to time and space is "linear".

In [ ]:
# Import interpolation function for steady flow field
from ipynb.fs.defs.Interpolant import interpolant_steady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
lap_U[np.isnan(lap_U)] = 0
lap_V[np.isnan(lap_V)] = 0
lap_W[np.isnan(lap_W)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_steady(X, Y, Z, lap_U, lap_V, lap_W)

Interpolant_lap_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_lap_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_lap_w = Interpolant[2] # RectangularBivariateSpline-object

# Instantaneous linear momentum barriers from active Trajectory Rotation Average $ (\mathrm{a\overline{TRA}}) $.

We visualize linear momentum barriers by applying the $ \mathrm{\overline{TRA}} $ to trajectories of instantaneous linear momentum barrier equation \ref{eq: linearmoment2}. As soon as the $ \mathrm{\overline{TRA}} $-field is applied to an active vector field, such as linear momentum, we refer to it as $ \mathrm{a\overline{TRA}} $ (active $ \mathrm{\overline{TRA}} $).

## active $ \mathrm{\overline{TRA}} $

We now compute the active $ \mathrm{\overline{TRA}} $ for the instantaneous linear momentum barrier equations. First, we evaluate the velocity along the particle trajectories from the autonomous ODE (eq. \ref{eq: linearmoment2}) for all initial conditions on the 2D cross-section. From the velocity of the trajectories (=time-derivative of flow map) we then compute the $ \mathrm{\overline{TRA}} $.

In [ ]:
# Import function to compute trajectory/velocity along trajectory
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import function to compute Trajectory Rotation Angle (TRA)
from ipynb.fs.defs.TRA import _TRA

# Import package for parallel computing
from joblib import Parallel, delayed

def compute_aTRA(x0, y0, z0, s_dummy, s):
    
    X0 = np.array([x0, y0, z0]) # array (3, Nx*Ny*Nz)
    
    DFDt = integration_dFdt(s_dummy, X0, X, Y, Z, Interpolant_lap_u, Interpolant_lap_v, Interpolant_lap_w, periodic, bool_unsteady)[1] # array (Nt, 3, Nx*Ny)
    
    aTRA = np.zeros((len(s), DFDt.shape[-1]))
    
    counter = 0
    for idx in np.searchsorted(s_dummy, s):
        
        for i in range(DFDt.shape[2]):
            
            aTRA[counter, i] = _TRA(DFDt[:idx+1,:,i].transpose(), s_dummy[idx])
            
        counter += 1
    
    return aTRA

# Split x0,y0,z0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0 = X_domain.ravel() # array (Nx*Ny, )
y0 = Y_domain.ravel() # array (Nx*Ny, )
z0 = z_section*np.ones(x0.shape[0],) # array (Nx*Ny, )

x0_batch = list(split(x0, Ncores)) # list (Nx*Ny)
y0_batch = list(split(y0, Ncores)) # list (Nx*Ny)
z0_batch = list(split(z0, Ncores)) # list (Nx*Ny)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_aTRA)(x0_batch[i], y0_batch[i], z0_batch[i], s_dummy, s) for i in range(len(x0_batch)))

# Extract results of aTRA computation on z=
aTRA = results[0]
for res in results[1:]:
    aTRA = np.append(aTRA, res, axis = -1)
    
aTRA = np.array(aTRA).reshape(len(s),Ny,Nx) # array (3, Ny, Nx)

##### $ \mathrm{a\overline{TRA}} $ over different dummy time-intervals

In the following we visualize the instantaneous linear momentum barriers at time $ t_0 = 0 $ through the $ \mathrm{a\overline{TRA}} $ over different dummy time-intervals. Note that contrary to the classic $ \mathrm{a\overline{TRA}} $, the $ \mathrm{a\overline{TRA}} $ diagnostic is not limited by the temporal domain as the dummy time-interval can be chosen to be arbitrarily large. We explore the structures of the $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_i} $ over different dummy time-intervals $ s_i \in \lbrace 1,2,3 \rbrace $ at time $ t_0 $.

In [ ]:
# Import matplotlib
import matplotlib.pyplot as plt

# plot iterate over different dummy time-intervals and display aTRA field calculated over each time-interval
def plot_aTRA(i, ticks_colorbar):
    
    # create figure
    fig = plt.figure(dpi = 180)
    ax = plt.axes()
    
    # Fontsize
    font_size = 7
    
    # Title
    ax.set_title(r'$ \mathrm{a\overline{TRA}} $'+f'$_{{{0}}}^{{{s[i]}}}$', fontsize = font_size)
    
    # Contourplot
    cax = ax.contourf(X_domain, Y_domain, aTRA[i,:,:], levels = 400, cmap = "gist_rainbow_r")
    
    # Set labels
    ax.set_xlabel("x", fontsize = font_size)
    ax.set_ylabel("y", fontsize = font_size)
    
    # Set ticks
    ax.set_xticks(np.arange(xmin, xmax+0.01, 0.1))
    ax.set_yticks(np.arange(ymin, ymax+0.01, 0.1))
    
    # Set fontsize
    ax.xaxis.set_tick_params(labelsize=font_size)
    ax.yaxis.set_tick_params(labelsize=font_size)
    
    # Colorbar
    cbar = plt.colorbar(cax, ticks = ticks_colorbar, format='%.0e')
    cbar.ax.tick_params(labelsize=font_size)
    ax.set_aspect('equal')
    plt.show()

##### $ \mathrm{a\overline{TRA}}_{0,0}^{s_1} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{a\overline{TRA}}_{0,0}^{s_1} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aTRA(0, np.linspace(0, 7*10**4, 8))

##### $ \mathrm{a\overline{TRA}}_{0,0}^{s_2} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{a\overline{TRA}}_{0,0}^{s_2} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aTRA(1, np.linspace(0, 7*10**4, 8))

##### $ \mathrm{a\overline{TRA}}_{0,0}^{s_3} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{a\overline{TRA}}_{0,0}^{s_3} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aTRA(2, np.linspace(0, 7*10**4, 8))

The figures focus on momentum-based active elliptic structures in a section of a 3D turbulent channel flow. In 2D cross-sections of the flow, elliptic regions are characterized by local maxima of $ \mathrm{a\overline{TRA}} $. The $ \mathrm{a\overline{TRA}} $ provides a clear picture of the main vortices. The features become even more pronounced for longer s-times, revealing several small scale elliptic structures. For very long dummy time-horizon, however, trajectories might leave the domain where the velocity field (and hence also the laplacian of the velocity field) is well defined.

# References

[1] Haller, G., Katsanoulis, S., Holzner, M., Frohnapfel, B., & Gatti, D. (2020). Objective barriers to the transport of dynamically active vector fields. Journal of Fluid Mechanics, 905.

[2] Haller, G., Aksamit, N., & Encinas-Bartos, A. P. (2021). Quasi-objective coherent structure diagnostics from single trajectories. Chaos: An Interdisciplinary Journal of Nonlinear Science, 31(4), 043131.

[3] Notebook 9.7.8. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.